In [1]:
# Download CBDB sqlite
# !git clone https://github.com/cbdb-project/cbdb_sqlite.git
import requests
import os
# url = "https://hf-mirror.com/datasets/cbdb/cbdb-sqlite/resolve/main/latest.7z?download=true"
url = "https://huggingface.co/datasets/cbdb/cbdb-sqlite/resolve/main/latest.7z?download=true"
r = requests.get(url, verify=False)
os.makedirs("./cbdb_sqlite", exist_ok=True)
with open("./cbdb_sqlite/latest.7z", "wb") as f:
    f.write(r.content)

d:\anaconda\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hf-mirror.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
d:\anaconda\lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cdn-lfs-us-1.hf-mirror.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [2]:
# import glob
import os

latest_file = "./cbdb_sqlite/latest.7z"
latest_db = latest_file.split("/")[2].split(".")[0] + ".db"
print(latest_db)
if os.path.isfile(latest_db):
    os.remove(latest_db)

latest.db


In [3]:
# Explode latest database file
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive(latest_file).extractall(".")

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [4]:
# Create connection to database
import sqlite3
import pandas as pd
global CONN
database_file = os.path.basename(latest_file).split(".")[0]+".db"
CONN = sqlite3.connect(database_file)

In [5]:
sql_str="SELECT c_dy, COUNT(*) as count FROM BIOG_MAIN GROUP BY c_dy "
dycount_df = pd.read_sql_query(sql_str, CONN).drop_duplicates().dropna()
dycount_df

,c_dy,count
1,0.0,17543
2,1.0,111
3,2.0,139
4,3.0,25
5,4.0,31
...,...,...
71,76.0,1
72,77.0,461
73,78.0,14
74,81.0,4


In [6]:
# Obtain statistics from various dynasties
Tang = dycount_df[dycount_df["c_dy"]==6]["count"].values[0]
Five_Dynasties = 0
Five_Dynasties_id = [7, 34, 47, 48, 52, 49, 36, 75, 9, 8, 11, 38, 12, 13, 55, 10, 66]
for id in Five_Dynasties_id:
    Five_Dynasties += dycount_df[dycount_df["c_dy"]==id]["count"].values[0]
Song = dycount_df[dycount_df["c_dy"]==15]["count"].values[0]
Liao = dycount_df[dycount_df["c_dy"]==16]["count"].values[0]
Jin = dycount_df[dycount_df["c_dy"]==17]["count"].values[0]
Yuan = dycount_df[dycount_df["c_dy"]==18]["count"].values[0]
Ming = dycount_df[dycount_df["c_dy"]==19]["count"].values[0]
Qing = dycount_df[dycount_df["c_dy"]==20]["count"].values[0]
Minguo = dycount_df[dycount_df["c_dy"]==21]["count"].values[0]

data = {  
    'Dynasty': ['Tang', '5 Dynasties', 'Song', 'Liao', 'Jin', 'Yuan', 'Ming', 'Qing', 'Minguo'],  
    'Count': [Tang, Five_Dynasties, Song, Liao, Jin, Yuan, Ming, Qing, Minguo]  
}  
dyres_df = pd.DataFrame(data) 
dyres_df

,Dynasty,Count
0,Tang,53895
1,5 Dynasties,1909
2,Song,74360
3,Liao,339
4,Jin,706
5,Yuan,24706
6,Ming,221089
7,Qing,123244
8,Minguo,4682


In [7]:
dyres_df.to_csv('dynasty_statistic.csv', index=False)

In [8]:
# Read the content of old JavaScript files
with open('script.js', 'r') as file:  
    js_code = file.read()  
  
# Delete old data  
start_delimiter = 'const chart1Data = ['  
end_delimiter = '];// Chart 1 Data '
start_index = js_code.find(start_delimiter)  
end_index = js_code.find(end_delimiter) + len(end_delimiter)  
js_code = js_code[:start_index] + js_code[end_index:]
# new data
js_data = ',\n'.join(  
    f'{{ label: "{dyname}", value: {count} }}'  
    for dyname,count in zip(data["Dynasty"],data["Count"])  
)
new_data = f'{start_delimiter}\n{js_data}\n{end_delimiter}'  
  
js_code = new_data +'\n' + js_code  
  
# Write updated JavaScript file 
with open('script.js', 'w') as file:  
    file.write(js_code)   

In [9]:
# update the date
import datetime  
import re  

svg_path = os.path.join(os.getcwd(), 'images', 'svg2.svg')  
with open(svg_path, 'r') as file:  
    svg_content = file.read()  
  
# get current date  
current_time = datetime.datetime.now().strftime('%Y年%m月%d日') 
current_time_english = datetime.datetime.now().strftime('%B %d, %Y') 
  
updated_svg_content = re.sub(r'更新日期：\d+年\d+月\d+日', f'更新日期：{current_time}', svg_content)    
updated_svg_content = re.sub(r'Updated on: \w+ \d+, \d+', f'Updated on: {current_time_english}', updated_svg_content)
with open(svg_path, 'w') as file:  
    file.write(updated_svg_content) 

In [10]:
# close the db
CONN.close()